# CMD Inference

In [2]:
from openvino.runtime import Core, PartialShape
import numpy as np
import os
import cv2
import time

num_images=120
ie = Core()
model_xml = "IR_model_FP16/saved_model.xml"
model = ie.read_model(model=model_xml)
input_layer = next(iter(model.inputs))
output_layer = next(iter(model.outputs))
new_shape = PartialShape([num_images, 224, 224, 3])
model.reshape({input_layer.any_name: new_shape})
compiled_model = ie.compile_model(model=model, device_name="MYRIAD")
result_index = np.argmax(compiled_model)
n, h, w, c = input_layer.shape
images = np.zeros(shape=(n, h, w, c))
for e, i in enumerate(os.listdir('Resized_images_120/')):
    img_name = i.split(".")[0]+".tif"
    image = cv2.cvtColor(cv2.imread(filename=os.path.join('Resized_images_120/', img_name)), code=cv2.COLOR_BGR2RGB)
    if image.shape[:-1] != (h, w):
        log.warning("Image {} is resized from {} to {}".format(args.input[i], image.shape[:-1], (h, w)))
    image = cv2.resize(image, (w, h))
    images[e] = image

## Single Batch

In [ ]:
start = time.perf_counter()
request = compiled_model.create_infer_request()
t0 = time.perf_counter()
res = request.infer(inputs={input_layer.any_name: images[:num_images]})
t1 = time.perf_counter()
infer_time =(t1 - t0)
print(infer_time)

## Multiple batch

In [1]:
infer_time= []
start = time.perf_counter()
request = compiled_model.create_infer_request()
for _ in range(num_images):
    t0 = time.perf_counter()
    res = request.infer(inputs={input_layer.any_name: images[:num_images]})
    t1 = time.perf_counter()
    infer_tm =(t1 - t0)
    infer_time.append(infer_tm)  
    print(infer_time)

In [ ]:
sum_time = sum(infer_time)
tot_img = num_images*len(images)
print("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
print(f"seconds per image, FPS: {tot_img/sum_time:.2f}")